In [236]:
import sqlite3
import pandas as pd
import numpy as np

In [167]:
conn2 = sqlite3.connect(':memory:')
c2 = conn2.cursor()

In [168]:
ents = [(32592188, '0x3f789c6d47b6b29c58e0534a3e9c8c325c088b2411d1b7c76dfb3fcea7e8130b'), 
        (32592368, '0x6799085cec26aedb774a27554f7da7c5a4205e69737ff6e730fd5ab662081d6c'),
        (32592420, '0x82ea0cc9a3311c74c0b7d8ea896474eb4444d43edd6157d48708f2d62100b2ef')]

In [169]:
df = pd.DataFrame(data=ents, columns = ['blockNumber', 'transactionHash'])

In [170]:
df.to_csv('ping_data.csv', index=False)

In [171]:
pd_df = pd.read_csv('ping_data.csv')

In [172]:
pd_df

,blockNumber,transactionHash
0,32592188,0x3f789c6d47b6b29c58e0534a3e9c8c325c088b2411d1...
1,32592368,0x6799085cec26aedb774a27554f7da7c5a4205e69737f...
2,32592420,0x82ea0cc9a3311c74c0b7d8ea896474eb4444d43edd61...


In [173]:
one = (32592420, '0x82ea0cc9a3311c74c0b7d8ea896474eb4444d43edd6157d48708f2d62100b2ef')
two = (32592420, '0x82ea0cc9a3311c74c0b7d8ea896474eb4444d43edd6157d48708f2d62100b2fg')

True

In [175]:
# def data_in_ping_df(ping_data:tuple)-> bool:
#     global pd_df
#     return ping_data[1] in pd_df['transactionHash'].values

In [176]:
def add_to_ping_csv(ping_data:tuple) -> None:
    _, transactionHash = ping_data
    # Open the csv into a df
    pd_df = pd.read_csv('ping_data.csv')
    # Check to see if the data exists
    if not transactionHash in pd_df['transactionHash'].values:
        # Add ping_data to df
        pd_df.loc[len(pd_df)] = ping_data
    # Write df to CSF
    pd_df.to_csv('ping_data.csv', index=False)

In [177]:
add_to_ping_csv(one)

In [178]:
add_to_ping_csv(two)

In [179]:
pd_df =  pd.read_csv('ping_data.csv')

In [180]:
tuple(pd_df.iloc[-1].values)

(32592420,
 '0x82ea0cc9a3311c74c0b7d8ea896474eb4444d43edd6157d48708f2d62100b2fg')

In [181]:
def get_last_entry() -> tuple:
    pd_df = pd.read_csv('ping_data.csv')
    return tuple(pd_df.iloc[-1].values)


In [182]:
last_blockNumber, _ = get_last_entry()

In [183]:
last_blockNumber

32592420

In [184]:
from listner import pingpong_contract, web3
from hexbytes import HexBytes 

In [185]:
ping_hash = HexBytes('0x82ea0cc9a3311c74c0b7d8ea896474eb4444d43edd6157d48708f2d62100b2ef')

In [186]:
# pingpong_contract.functions.pong('0x82ea0cc9a3311c74c0b7d8ea896474eb4444d43edd6157d48708f2d62100b2ef').build_transaction({'nonce': 10})

# tx = greeter.functions.greet("newGreet").buildTransaction({'nonce': web3.eth.getTransactionCount('your accountaddress')}}

# tx = pingpong_contract.functions.pong('0x82ea0cc9a3311c74c0b7d8ea896474eb4444d43edd6157d48708f2d62100b2ef').transact({'nonce': 10})


In [188]:
# web3.eth.estimateGas(signed_tx)

In [189]:
'''
num = ...
# build transaction
build_tx = contract.functions.set(num).buildTransaction({
    'from': ...,
    'nonce': ...,
    'gas': ...,
    'gasPrice': ...
})
# sign transaction
sign_tx = w3.eth.account.signTransaction(build_tx, private_key)
# send the transaction
tx_hash = w3.eth.sendRawTransaction(sign_tx.rawTransaction)

'''



"\nnum = ...\n# build transaction\nbuild_tx = contract.functions.set(num).buildTransaction({\n    'from': ...,\n    'nonce': ...,\n    'gas': ...,\n    'gasPrice': ...\n})\n# sign transaction\nsign_tx = w3.eth.account.signTransaction(build_tx, private_key)\n# send the transaction\ntx_hash = w3.eth.sendRawTransaction(sign_tx.rawTransaction)\n\n"

In [271]:
ping_hash = '0x82ea0cc9a3311c74c0b7d8ea896474eb4444d43edd6157d48708f2d62100b2ef'

build_tx = pingpong_contract.functions.pong(ping_hash).buildTransaction({
        'from':'00000',
        'nonce':0,
        'gas': 100000,
        'gasPrice':web3.eth.gas_price
    })

print(build_tx)

{'value': 0, 'chainId': 42, 'from': '00000', 'nonce': 0, 'gas': 100000, 'gasPrice': 2500000007, 'to': '0x7D3a625977bFD7445466439E60C495bdc2855367', 'data': '0x05ba79a282ea0cc9a3311c74c0b7d8ea896474eb4444d43edd6157d48708f2d62100b2ef'}


In [191]:
# Use this to estimate gas
# web3.eth.estimate_gas(build_tx)

In [192]:
web3.eth.gas_price

5000000000

In [193]:
import time

LOCKED = False

def wait_for_lock(func):
    ''' Decorator function to ensure that Ping & Pong 
        don't write to CSV file at the same time'''
    def wrapper(*args, **kwargs):
        global LOCKED
        if not LOCKED:
            LOCKED = True
            func(*args, **kwargs)
            LOCKED = False
        else:
            for i in range(10):
                if not LOCKED:
                    LOCKED = True
                    func(*args, **kwargs)
                    LOCKED = False
                else:
                    time.sleep(2)
    return wrapper 

@wait_for_lock
def ping(sleep:int):
    time.sleep(sleep)
    print("Ping")

@wait_for_lock
def pong(sleep:int):
    time.sleep(sleep)
    print("Pong")




In [254]:
pigd = (32592420, '0x82ea0cc9a3311c74c0b7d8ea896474eb4444d43edd6157d48708f2d62100b2fg')

pdff = pd.DataFrame(columns = ['blockNumber', 'transactionHash', 'PongStatus'])

In [255]:
len(pdff)

0

In [256]:
inp = list(pigd)
inp.append(0)
inp


[32592420,
 '0x82ea0cc9a3311c74c0b7d8ea896474eb4444d43edd6157d48708f2d62100b2fg',
 0]

In [257]:
pdff.loc[len(pdff)] = inp
pdff

# pdff.append({col:i for (col, i) in zip(pdff.columns, inp)}, ignore_index=True)

,blockNumber,transactionHash,PongStatus
0,32592420,0x82ea0cc9a3311c74c0b7d8ea896474eb4444d43edd61...,0


In [258]:
pdff

,blockNumber,transactionHash,PongStatus
0,32592420,0x82ea0cc9a3311c74c0b7d8ea896474eb4444d43edd61...,0


In [259]:
another = [32623896, '0x7b8a0f3eb531143840662fe059356486ad5d9c1c2838aec40ad58395a90e5269', 0]

In [260]:
pdff.loc[len(pdff)] = another
pdff

,blockNumber,transactionHash,PongStatus
0,32592420,0x82ea0cc9a3311c74c0b7d8ea896474eb4444d43edd61...,0
1,32623896,0x7b8a0f3eb531143840662fe059356486ad5d9c1c2838...,0


In [267]:
# def get_first_unPonged():
# Read csv
# Returns the first index of unPonged
index = pdff.loc[pdff['PongStatus'] == 0].head(1).index[0]
index

1

In [269]:
pdff.iloc[index]['transactionHash']
# Do something -> send a Pong

'0x7b8a0f3eb531143840662fe059356486ad5d9c1c2838aec40ad58395a90e5269'

In [265]:
pdff.loc[index,'PongStatus'] = 1

In [266]:
pdff

,blockNumber,transactionHash,PongStatus
0,32592420,0x82ea0cc9a3311c74c0b7d8ea896474eb4444d43edd61...,1
1,32623896,0x7b8a0f3eb531143840662fe059356486ad5d9c1c2838...,0


In [270]:
type(index)

numpy.int64

In [275]:
# from pong_transact import tx_hash
import datetime

In [279]:

counter_start = None
max_time = 5

def test():
    counter_start = datetime.datetime.now()
    time.sleep(10)
    current_time = datetime.datetime.now()
    if current_time > counter_start + datetime.timedelta(seconds=max_time):
        print("Taking too long")
    






In [280]:
test()

Taking too long
